# Problem Statement:

You want to collect text data using Twitter API.

https://developer.twitter.com/en

# AIML Batch2:

__Consumer API keys__

__API key:__

nucwiQk9GtsJfWFSGQjXFicnq

__API secret key:__

psbIvsTINgr6eEu89jMe2FwsOM28Cs3KaVLWUfBjADkAfFfhZV

__Access token:__

296102538-4rqAPJBZSl2dQW2Wd1qvciWfkNKFpv9GZ3IBp5t7

__Access token secret:__

FoCmyrWsWVeyxjksdRwqujVyzQGT9tZVjRGqJoav021MU

https://anaconda.org/conda-forge/tweepy

https://anaconda.org/anaconda/openpyxl

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import json

In [2]:
import tweepy
import openpyxl
from tweepy import OAuthHandler

In [3]:
# credentials

consumer_key = "nucwiQk9GtsJfWFSGQjXFicnq"
consumer_secret = "psbIvsTINgr6eEu89jMe2FwsOM28Cs3KaVLWUfBjADkAfFfhZV"
access_token = "296102538-4rqAPJBZSl2dQW2Wd1qvciWfkNKFpv9GZ3IBp5t7"
access_token_secret = "FoCmyrWsWVeyxjksdRwqujVyzQGT9tZVjRGqJoav021MU"

In [4]:
# call API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [5]:
# Lets provide the query which I need to search

query = "US election Joe Biden"

# Fetch tweets

Tweets = api.search(query, count=10, lang= 'en', exclude= 'retweets', tweet_mode= 'extended')

for tweet in Tweets:
    print(tweet)
    print("=================================================")

Status(_api=<tweepy.api.API object at 0x0000022C324EECA0>, _json={'created_at': 'Mon Jan 25 12:14:04 +0000 2021', 'id': 1353677513797611520, 'id_str': '1353677513797611520', 'full_text': "@Murendi UK done with us long time ago,so they following in their footsteps. Election of President Joe Biden based on American voter's needs who rejected former President Trump convincingly. So it shouldn't raise any eyebrows knowing he's implementing what's suitable to his country", 'truncated': False, 'display_text_range': [9, 282], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'Murendi', 'name': 'Murendeni Mafumo', 'id': 41323575, 'id_str': '41323575', 'indices': [0, 8]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': 1353674745238138882, 'in_reply_to_status_id_str': '1353674745238138882', 'in_reply_to_user_id': 4

# Getting the Tweets and some of its related Attributes

In [77]:
df = pd.DataFrame(columns=[
    'Tweets', 'User', 'User_statuses_count',
    'user_followers', 'User_location', 'User_verified',
    'fav_count', 'rt_count', 'tweet_date'
])
df

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date


In [78]:
def stream(data, file_name):
    i=0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, lang='en').items():
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        df.loc[i, 'user_followers'] = tweet.user.followers_count
        df.loc[i, 'User_location'] = tweet.user.location
        df.loc[i, 'User_verified'] = tweet.user.verified
        df.loc[i, 'fav_count'] = tweet.favorite_count
        df.loc[i, 'rt_count'] = tweet.retweet_count
        df.loc[i, 'tweet_date'] = tweet.created_at
        df.to_excel('{}.xlsx'.format(file_name))
        i = i+1
        if i == 1000:
            break
        else:
            pass

In [79]:
stream(data=["Arnab Goswami’s leaked WhatsApp chats"], file_name='my_tweets')

In [80]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date
0,RT @karanbirtinna: Wow Arnab Goswami's WhatsAp...,Discombobulated.,314381,375,North-Mid Earth,False,0,17,2021-01-24 13:59:12
1,Many believe that anyone can snoop into one's ...,Praveen Kumar,130,60,🇮🇳,False,1,0,2021-01-24 08:41:39
2,RT @dhruv_rathee: More than 500 pages of Arnab...,Samir Khan,9735,51,"Jamnagar, India",False,0,5461,2021-01-24 05:54:38
3,RT @AnuragK35539521: Leaked WhatsApp chats bet...,surekha gupta,9681,2371,"Ahmadabad City, India",False,0,3,2021-01-24 03:50:36
4,RT @LogicalIndians: One week has passed since ...,Raj Grewal,5168,186,Germany,False,0,5,2021-01-23 21:08:07


In [81]:
df.shape

(946, 9)

In [82]:
from textblob import TextBlob

In [83]:
import re
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [84]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity>0:
        return 'Positive'
    elif analysis.sentiment.polarity==0:
        return 'Neutral'
    else:
        return 'Negative'

In [85]:
df['clean_tweet'] = df['Tweets'].apply(lambda x: clean_tweet(x))

In [86]:
df[['Tweets','clean_tweet']].head()

,Tweets,clean_tweet
0,RT @karanbirtinna: Wow Arnab Goswami's WhatsAp...,RT Wow Arnab Goswami s WhatsApp chats leaked a...
1,Many believe that anyone can snoop into one's ...,Many believe that anyone can snoop into one s ...
2,RT @dhruv_rathee: More than 500 pages of Arnab...,RT rathee More than 500 pages of Arnab Goswami...
3,RT @AnuragK35539521: Leaked WhatsApp chats bet...,RT Leaked WhatsApp chats between BARC Ceo amp ...
4,RT @LogicalIndians: One week has passed since ...,RT One week has passed since Editor in chief A...


In [87]:
df['Sentiment'] = df['clean_tweet'].apply(lambda x: analyze_sentiment(x))

In [88]:
df[['clean_tweet','Sentiment']].head()

,clean_tweet,Sentiment
0,RT Wow Arnab Goswami s WhatsApp chats leaked a...,Positive
1,Many believe that anyone can snoop into one s ...,Positive
2,RT rathee More than 500 pages of Arnab Goswami...,Positive
3,RT Leaked WhatsApp chats between BARC Ceo amp ...,Neutral
4,RT One week has passed since Editor in chief A...,Negative


In [89]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [90]:
df[['clean_tweet','Sentiment']].head()

,clean_tweet,Sentiment
0,RT Wow Arnab Goswami s WhatsApp chats leaked a...,Positive
1,Many believe that anyone can snoop into one s ...,Positive
2,RT rathee More than 500 pages of Arnab Goswami...,Positive
3,RT Leaked WhatsApp chats between BARC Ceo amp ...,Neutral
4,RT One week has passed since Editor in chief A...,Negative


In [91]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet,Sentiment
0,RT @karanbirtinna: Wow Arnab Goswami's WhatsAp...,Discombobulated.,314381,375,North-Mid Earth,False,0,17,2021-01-24 13:59:12,RT Wow Arnab Goswami s WhatsApp chats leaked a...,Positive
1,Many believe that anyone can snoop into one's ...,Praveen Kumar,130,60,🇮🇳,False,1,0,2021-01-24 08:41:39,Many believe that anyone can snoop into one s ...,Positive
2,RT @dhruv_rathee: More than 500 pages of Arnab...,Samir Khan,9735,51,"Jamnagar, India",False,0,5461,2021-01-24 05:54:38,RT rathee More than 500 pages of Arnab Goswami...,Positive
3,RT @AnuragK35539521: Leaked WhatsApp chats bet...,surekha gupta,9681,2371,"Ahmadabad City, India",False,0,3,2021-01-24 03:50:36,RT Leaked WhatsApp chats between BARC Ceo amp ...,Neutral
4,RT @LogicalIndians: One week has passed since ...,Raj Grewal,5168,186,Germany,False,0,5,2021-01-23 21:08:07,RT One week has passed since Editor in chief A...,Negative


In [92]:
n = 20
print('Original Tweet:\n'+df['Tweets'][n])
print()
print('Clean Tweet:\n'+df['clean_tweet'][n])
print()
print("Sentiment:\n"+df['Sentiment'][n])

Original Tweet:
RT @dhruv_rathee: More than 500 pages of Arnab Goswami's WhatsApp chat has been leaked! 

Not only is there clear cut evidence of abuse of…

Clean Tweet:
RT rathee More than 500 pages of Arnab Goswami s WhatsApp chat has been leaked Not only is there clear cut evidence of abuse of

Sentiment:
Positive


In [93]:
n = 202
print('Original Tweet:\n'+df['Tweets'][n])
print()
print('Clean Tweet:\n'+df['clean_tweet'][n])
print()
print("Sentiment:\n"+df['Sentiment'][n])

Original Tweet:
The leaked WhatsApp chats of Arnab Goswami, an anchorperson of Indian Republic TV, exposed the sinister designs hat… https://t.co/F3vXQYmA6o

Clean Tweet:
The leaked WhatsApp chats of Arnab Goswami an anchorperson of Indian Republic TV exposed the sinister designs hat

Sentiment:
Negative


In [94]:
n = 50
print('Original Tweet:\n'+df['Tweets'][n])
print()
print('Clean Tweet:\n'+df['clean_tweet'][n])
print()
print("Sentiment:\n"+df['Sentiment'][n])

Original Tweet:
RT @RiaRevealed: Republic TV's Arnab Goswami has been slammed by the grieving kin of Pulwama attack martyrs. They demanded strict action ag…

Clean Tweet:
RT Republic TV s Arnab Goswami has been slammed by the grieving kin of Pulwama attack martyrs They demanded strict action ag

Sentiment:
Positive


In [95]:
df[df.Sentiment=='Positive'].shape[0]

246

In [96]:
df[df.Sentiment=='Neutral'].shape[0]

346

In [97]:
df[df.Sentiment=='Negative'].shape[0]

354